In [2]:
!pip list

Package                            Version             Editable project location
---------------------------------- ------------------- -------------------------
absl-py                            1.4.0
accelerate                         1.5.2
aiofiles                           22.1.0
aiohappyeyeballs                   2.6.1
aiohttp                            3.11.18
aiosignal                          1.3.2
aiosqlite                          0.21.0
alabaster                          1.0.0
albucore                           0.0.23
albumentations                     2.0.5
ale-py                             0.10.2
alembic                            1.15.2
altair                             5.5.0
annotated-types                    0.7.0
annoy                              1.17.3
ansicolors                         1.1.8
antlr4-python3-runtime             4.9.3
anyio                              4.9.0
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
args    

In [1]:
#!pip install -q bitsandbytes accelerate
#!pip install -q -U git+https://github.com/fabienfrfr/tptt@main
!pip install -q git+https://github.com/fabienfrfr/tptt@dev

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.5 MB/s eta 0:00:00:00:0100:01


In [2]:
# only in kaggle for HF
from huggingface_hub import login, HfApi
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
login(token=hf_token)
api = HfApi()

In [3]:
QUANTIZATION = False

MODEL_TYPE = 'llama'

# out_proj can doing NaN ?
if MODEL_TYPE == 'llama':
    base_model_name="meta-llama/Llama-3.2-1B"
    base_tokenizer_name=base_model_name
    target_modules = ["q_proj","k_proj","v_proj","o_proj"]  # Llama, Mistral, OLMo. Minimal : q_proj, v_proj
elif MODEL_TYPE == 'olmo':
    base_model_name="allenai/OLMo-1B-hf"
    base_tokenizer_name=base_model_name
    target_modules = ["q_proj","k_proj","v_proj","o_proj"]  # Llama, Mistral, OLMo. Minimal : q_proj, v_proj
elif MODEL_TYPE == "elm":
    base_model_name="apple/OpenELM-1_1B"
    base_tokenizer_name="meta-llama/Llama-2-7b-hf" # for openELM only
    target_modules = ["qkv_proj","out_proj"]  # OpenELM,
elif MODEL_TYPE == 'qwen':
    base_model_name="Qwen/Qwen2.5-1.5B"
    base_tokenizer_name=base_model_name
    target_modules = ["q_proj","k_proj","v_proj","o_proj"]  # Llama, Mistral, OLMo. Minimal : q_proj, v_proj
elif MODEL_TYPE == 'mistral':
    base_model_name="NickyNicky/experimental-Mistral-1b-V00"
    base_tokenizer_name=base_model_name
    target_modules = ["q_proj","k_proj","v_proj","o_proj"]  # Llama, Mistral, OLMo. Minimal : q_proj, v_proj


max_chunk_size = 64 # 64

# basic training
N = 500 # 500
EPOCH = 5 # 5
DATASET = "yahma/alpaca-cleaned"

#target_modules = ["c_attn","c_proj"]  # GPT-2

# saving
username = "ffurfaro"
model_name = '/Titans-' + base_model_name.split('/')[1]
dir_path = '.' + model_name
repo_id = username + '/' + model_name.lstrip('/')

In [4]:
import os
import shutil
import json

if QUANTIZATION:
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

from transformers import (
    AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, 
    DataCollatorForLanguageModeling, TrainerCallback
)
from datasets import load_dataset
from peft import LoraConfig, PeftConfig, get_peft_model

if QUANTIZATION:
    from transformers import BitsAndBytesConfig
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    )
else :
    bnb_config = None

# Tools
import tptt

2025-06-13 18:46:36.284044: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749840396.470409      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749840396.525957      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
).to_dict()

In [6]:
## Transforming into Titans
config = tptt.TpttConfig(
    base_model_name=base_model_name,
    max_self_attn_length = 8192,
    max_chunk_size = max_chunk_size,
    lora_config=lora_config,
)
display(config)

model = tptt.TpttModel(config,
    trust_remote_code=True,
    attn_implementation="eager",
    token=hf_token,
    torch_dtype=torch.bfloat16, # for model trained in float32 !
    quantization_config=bnb_config,)
display(model)
model.backbone.print_trainable_parameters()

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

TpttConfig {
  "architectures": [
    "TpttModel"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "configuration_tptt.TpttConfig",
    "AutoModelForCausalLM": "modeling_tptt.TpttModel"
  },
  "base_model_name": "meta-llama/Llama-3.2-1B",
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "lora_config": {
    "alpha_pattern": {},
    "auto_mapping": null,
    "base_model_name_or_path": null,
    "bias": "none",
    "eva_config": null,
    "exclude_modules": null,
    "fan_in_fan_out": false,
    "inference_mode": false,
    "init_lora_weights": true,
    "layer_replication": null,
    "layers_pattern": null,
    "layers_to_transform": null,
    "loftq_config": {},
    "lora_alpha": 16,
    "lora_bias": false,
    "lora_dropout": 0.05,
    "megatron_config": null,
    "megatron_core": "megatron.core",
    "modules_to_save": null,
    "peft_type": "LORA",
    "r": 8,
   

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

TpttModel(
  (backbone): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): LlamaForCausalLM(
        (model): LlamaModel(
          (embed_tokens): Embedding(128256, 2048)
          (layers): ModuleList(
            (0-15): 16 x LlamaDecoderLayer(
              (self_attn): LiZAttention(
                (base_attn): LlamaAttention(
                  (q_proj): lora.Linear(
                    (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=2048, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_emb

trainable params: 1,703,936 || all params: 1,237,518,336 || trainable%: 0.1377


In [7]:
tokenizer = AutoTokenizer.from_pretrained(base_tokenizer_name, token=hf_token)
# Ensure the tokenizer has a padding token for batching
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token or "[PAD]"

raw_dataset = load_dataset(DATASET)["train"].select(range(N))

def preprocess_fn(samples):
    """
    Tokenize the samples for causal language modeling.
    Concatenate instruction, input, and output as needed.
    """
    prompts = [
        f"{instr}\n{inp}" if inp else instr
        for instr, inp in zip(samples["instruction"], samples["input"])
    ]
    # Optionally, append output for supervised fine-tuning
    prompts = [f"{p}\n{out}" for p, out in zip(prompts, samples["output"])]
    tokens = tokenizer(
        prompts,
        truncation=True,
        max_length=384, #256, 512
        #padding= "max_length",
        padding="longest",
        return_attention_mask=True,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = raw_dataset.map(
    preprocess_fn, batched=True, remove_columns=raw_dataset.column_names
)

# Tokenize the dataset in batches and remove original columns
tokenized_dataset = raw_dataset.map(
    preprocess_fn, batched=True, remove_columns=raw_dataset.column_names)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [8]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Step 6: Define HuggingFace TrainingArguments for reproducible training
training_args = TrainingArguments(
    output_dir=dir_path, #"./tptt_output",
    per_device_train_batch_size=3, # per_device_train_batch_size * N GPU --> VRAM limit risk 
    num_train_epochs=EPOCH,
    learning_rate=  5e-4,
    max_grad_norm=1.0, # gradiant clipping --> limit overflow gradiant
    bf16=True,  # Use mixed precision if supported by hardware --> limit underflow gradiant : same range of float32
    ddp_find_unused_parameters=False, 
    logging_steps=5,
    save_total_limit=0,  # Limit HDD
    seed=42,
    save_strategy="epoch",
    report_to="tensorboard",
)

# LiZA MaG callback
initial_weight=0.01,
final_weight=0.5,
transition_step=100,
liza_callback = tptt.AdjustMaGWeightCallback(
            model,
            initial_weight=initial_weight,
            final_weight=final_weight,
            transition_step=transition_step,)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset, # add test dataset ?
    data_collator=data_collator,
    processing_class=tokenizer,
    callbacks=[liza_callback],
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
5,1.832600
10,1.866700
15,1.761400
20,1.623900
25,1.285200
30,1.605700
35,1.700300
40,1.528700
45,1.368800
50,1.523100


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=420, training_loss=1.4463425431932722, metrics={'train_runtime': 1654.117, 'train_samples_per_second': 1.511, 'train_steps_per_second': 0.254, 'total_flos': 5615136276480000.0, 'train_loss': 1.4463425431932722, 'epoch': 5.0, 'mag_weight': 0.4951})

In [9]:
def remove_checkpoint_folders(dir_path):
    for item in os.listdir(dir_path):
        item_path = os.path.join(dir_path, item)
        if os.path.isdir(item_path) and item.startswith("checkpoint-"):
            shutil.rmtree(item_path)
            print(f"Deleted : {item_path}")

remove_checkpoint_folders(dir_path)

Deleted : ./Titans-Llama-3.2-1B/checkpoint-168
Deleted : ./Titans-Llama-3.2-1B/checkpoint-336
Deleted : ./Titans-Llama-3.2-1B/checkpoint-252
Deleted : ./Titans-Llama-3.2-1B/checkpoint-420
Deleted : ./Titans-Llama-3.2-1B/checkpoint-84


In [10]:
tokenizer.save_pretrained(dir_path)
model.save_pretrained(dir_path)

In [11]:
last_log = trainer.state.log_history[-2]
train_log = trainer.state.log_history[-1]
metrics = {
    "loss": last_log.get("loss", train_log.get("train_loss", "N/A")),
    "learning_rate": last_log.get("learning_rate", "N/A"),
    "epochs": train_log.get("epoch", "N/A"),
    "train_runtime": train_log.get("train_runtime", "N/A"),
    "train_samples_per_second": train_log.get("train_samples_per_second", "N/A"),
    "train_steps_per_second": train_log.get("train_steps_per_second", "N/A"),
    "total_flos": train_log.get("total_flos", "N/A"),
    "grad_norm": last_log.get("grad_norm", "N/A"),
}

tptt.generate_model_card(
    path=dir_path,
    config=model.config,
    batch_size=training_args.per_device_train_batch_size,
    dataset=DATASET,
    platform="Kaggle",
    hardware="NVIDIA 2xT4",
    **metrics
)

In [12]:
api.create_repo(
    repo_id=repo_id,
    token=hf_token,
    repo_type="model",
    exist_ok=True,
    private=False
)
api.upload_folder(
    folder_path=dir_path,
    repo_id=repo_id,
    repo_type="model",
    token=hf_token,
    commit_message="Upload model + init tptt code"
)

adapter_model.safetensors:   0%|          | 0.00/6.83M [00:00<?, ?B/s]

events.out.tfevents.1749840431.c24f68c16607.35.0:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.48G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ffurfaro/Titans-Llama-3.2-1B/commit/52a0fe84d5792d32b2510641edcdef430dab0495', commit_message='Upload model + init tptt code', commit_description='', oid='52a0fe84d5792d32b2510641edcdef430dab0495', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ffurfaro/Titans-Llama-3.2-1B', endpoint='https://huggingface.co', repo_type='model', repo_id='ffurfaro/Titans-Llama-3.2-1B'), pr_revision=None, pr_num=None)

In [13]:
from transformers import AutoModelForCausalLM, AutoConfig

#config = AutoConfig.from_pretrained(repo_id, model_type="tptt", trust_remote_code=True)
model_tptt = AutoModelForCausalLM.from_pretrained(
    repo_id, 
    #config=config,
    trust_remote_code=True
)
display(model_tptt)

config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

configuration_tptt.py:   0%|          | 0.00/4.66k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ffurfaro/Titans-Llama-3.2-1B:
- configuration_tptt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_tptt.py:   0%|          | 0.00/29.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ffurfaro/Titans-Llama-3.2-1B:
- modeling_tptt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/6.83M [00:00<?, ?B/s]

Some weights of TpttModel were not initialized from the model checkpoint at ffurfaro/Titans-Llama-3.2-1B and are newly initialized: ['backbone.base_model.model.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

TpttModel(
  (backbone): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): LlamaForCausalLM(
        (model): LlamaModel(
          (embed_tokens): Embedding(128256, 2048)
          (layers): ModuleList(
            (0-15): 16 x LlamaDecoderLayer(
              (self_attn): LiZAttention(
                (base_attn): LlamaAttention(
                  (q_proj): lora.Linear(
                    (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=2048, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_emb

In [14]:
# Prepare for inference
device = 0 if torch.cuda.is_available() else -1
model_tptt.to(f"cuda:{device}" if device != -1 else "cpu")

model_tptt.eval()
pipe = tptt.TpttPipeline(model=model_tptt, tokenizer=tokenizer, device=device)

# Generate text
result = pipe("Bonjour, I'm Fabien Furfaro,", max_new_tokens=100)
print(result[0]["generated_text"])

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Bonjour, I'm Fabien Furfaro, I'm a French programmer and I'm currently working on a project that involves AI and machine learning. I'm interested in the field of AI and I'm always looking for new ways to improve my programming skills. Feel free to contact me if you have


## Test

In [37]:
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import os

def tflog2pandas(path):
    runlog_data = pd.DataFrame({"metric": [], "value": [], "step": []})
    try:
        event_acc = EventAccumulator(path)
        event_acc.Reload()
        tags = event_acc.Tags()["scalars"]
        for tag in tags:
            event_list = event_acc.Scalars(tag)
            values = list(map(lambda x: x.value, event_list))
            step = list(map(lambda x: x.step, event_list))
            r = {"metric": [tag] * len(step), "value": values, "step": step}
            r = pd.DataFrame(r)
            runlog_data = pd.concat([runlog_data, r])
    except Exception as e:
        print(f"Erreur sur le fichier {path}: {e}")
    return runlog_data

# Chemin dossier de logs TensorBoard
run_files = os.listdir(dir_path + "/runs")
print(run_files)
for r in run_files :
    log_dir = dir_path + "/runs/" + r
    tfevents_files = [f for f in os.listdir(log_dir) if f.startswith("events.out.tfevents.")]
    if tfevents_files:
        df = tflog2pandas(os.path.join(log_dir, tfevents_files[0]))
    else:
        print("Aucun fichier tfevents trouvé dans", log_dir)
    display(df.metric.unique())

['Jun09_14-50-18_c0566a46a209', 'Jun09_14-40-05_c0566a46a209', 'Jun09_14-27-27_c0566a46a209', 'Jun09_14-48-52_c0566a46a209']


array(['mag_weight'], dtype=object)

array(['train/loss', 'train/grad_norm', 'train/learning_rate',
       'train/epoch', 'train/train_runtime',
       'train/train_samples_per_second', 'train/train_steps_per_second',
       'train/total_flos', 'train/train_loss'], dtype=object)

array(['train/loss', 'train/grad_norm', 'train/learning_rate',
       'train/epoch', 'train/train_runtime',
       'train/train_samples_per_second', 'train/train_steps_per_second',
       'train/total_flos', 'train/train_loss'], dtype=object)

array(['mag_weight'], dtype=object)